In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, LSTM, GRU, Dense, Concatenate, Conv1D, GlobalMaxPooling1D, MultiHeadAttention, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from bayes_opt import BayesianOptimization

# Data Preprocessing
def preprocess_data(df):
    event_type_mapping = df['event_type'].astype('category').cat.categories
    agent_id_mapping = df['agent_id'].astype('category').cat.categories
    context_mapping = df['context'].astype('category').cat.categories

    df['event_type'] = df['event_type'].astype('category').cat.codes
    df['agent_id'] = df['agent_id'].astype('category').cat.codes
    df['context'] = df['context'].astype('category').cat.codes

    return df, event_type_mapping, agent_id_mapping, context_mapping

# Load your dataset
df = pd.read_csv('1k_single_agent_minmax.csv')
df, event_type_mapping, agent_id_mapping, context_mapping = preprocess_data(df)

# Define constants
num_classes = df['event_type'].nunique()
num_agents = df['agent_id'].nunique()
num_contexts = df['context'].nunique()
sequence_length = 10  # Adjust based on your data

# Create sequences for training
def create_sequences(df, sequence_length):
    sequences = []
    for i in range(len(df) - sequence_length):
        seq = df.iloc[i:i+sequence_length].values
        sequences.append(seq)
    return np.array(sequences)

sequences = create_sequences(df, sequence_length)

# Splitting the data
train_size = int(0.7 * len(sequences))
val_size = int(0.15 * len(sequences))
train_sequences = sequences[:train_size]
val_sequences = sequences[train_size:train_size+val_size]
test_sequences = sequences[train_size+val_size:]

# Extract targets
def extract_targets(sequences):
    next_event = to_categorical(sequences[:, -1, 0], num_classes=num_classes)
    agent = to_categorical(sequences[:, -1, 1], num_classes=num_agents)
    context = to_categorical(sequences[:, -1, 2], num_classes=num_contexts)
    anomaly = np.random.randint(0, 2, size=(sequences.shape[0], 1))  # Placeholder for anomaly
    return next_event, agent, context, anomaly

train_targets = extract_targets(train_sequences)
val_targets = extract_targets(val_sequences)
test_targets = extract_targets(test_sequences)

# Define model branches
def lstm_branch(input_shape, units):
    inputs = Input(shape=(input_shape[0],))
    x = Embedding(input_dim=num_classes, output_dim=64)(inputs)
    x = LSTM(units, return_sequences=False)(x)
    return inputs, x

def gru_branch(input_shape, units):
    inputs = Input(shape=(input_shape[0],))
    x = Embedding(input_dim=num_classes, output_dim=64)(inputs)
    x = GRU(units, return_sequences=False)(x)
    return inputs, x

def transformer_branch(input_shape, num_heads):
    inputs = Input(shape=(input_shape[0],))
    x = Embedding(input_dim=num_classes, output_dim=64)(inputs)
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=64)(x, x)
    x = LayerNormalization()(x + attn_output)
    x = GlobalMaxPooling1D()(x)
    return inputs, x

def convnet_branch(input_shape, filters):
    inputs = Input(shape=(input_shape[0],))
    x = Embedding(input_dim=num_classes, output_dim=64)(inputs)
    x = Conv1D(filters, kernel_size=3, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)
    return inputs, x

# Build the model with hyperparameters
def build_model(sequence_length, lstm_units, gru_units, transformer_heads, conv_filters, dense_units, learning_rate):
    lstm_event_inputs, lstm_event_output = lstm_branch((sequence_length,), lstm_units)
    lstm_agent_inputs, lstm_agent_output = lstm_branch((sequence_length,), lstm_units)
    lstm_context_inputs, lstm_context_output = lstm_branch((sequence_length,), lstm_units)

    gru_event_inputs, gru_event_output = gru_branch((sequence_length,), gru_units)
    gru_agent_inputs, gru_agent_output = gru_branch((sequence_length,), gru_units)
    gru_context_inputs, gru_context_output = gru_branch((sequence_length,), gru_units)

    transformer_event_inputs, transformer_event_output = transformer_branch((sequence_length,), transformer_heads)
    transformer_agent_inputs, transformer_agent_output = transformer_branch((sequence_length,), transformer_heads)
    transformer_context_inputs, transformer_context_output = transformer_branch((sequence_length,), transformer_heads)

    convnet_event_inputs, convnet_event_output = convnet_branch((sequence_length,), conv_filters)
    convnet_agent_inputs, convnet_agent_output = convnet_branch((sequence_length,), conv_filters)
    convnet_context_inputs, convnet_context_output = convnet_branch((sequence_length,), conv_filters)

    # Concatenate outputs from all branches
    concatenated = Concatenate()([
        lstm_event_output, lstm_agent_output, lstm_context_output,
        gru_event_output, gru_agent_output, gru_context_output,
        transformer_event_output, transformer_agent_output, transformer_context_output,
        convnet_event_output, convnet_agent_output, convnet_context_output
    ])

    shared_dense = Dense(dense_units, activation='relu')(concatenated)

    next_event_head = Dense(num_classes, activation='softmax', name='next_event')(shared_dense)
    agent_head = Dense(num_agents, activation='softmax', name='agent')(shared_dense)
    context_head = Dense(num_contexts, activation='softmax', name='context')(shared_dense)
    anomaly_head = Dense(1, activation='sigmoid', name='anomaly')(shared_dense)

    model = Model(inputs=[
        lstm_event_inputs, lstm_agent_inputs, lstm_context_inputs,
        gru_event_inputs, gru_agent_inputs, gru_context_inputs,
        transformer_event_inputs, transformer_agent_inputs, transformer_context_inputs,
        convnet_event_inputs, convnet_agent_inputs, convnet_context_inputs
    ], outputs=[next_event_head, agent_head, context_head, anomaly_head])

    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss={'next_event': 'categorical_crossentropy', 
                        'agent': 'categorical_crossentropy',
                        'context': 'categorical_crossentropy', 
                        'anomaly': 'binary_crossentropy'},
                  loss_weights={'next_event': 1.0, 'agent': 0.5, 'context': 0.5, 'anomaly': 0.1},
                  metrics={'next_event': 'accuracy', 
                           'agent': 'accuracy',
                           'context': 'accuracy',
                           'anomaly': 'accuracy'})
    return model

# Define the objective function for Bayesian Optimization
def objective_function(lstm_units, gru_units, transformer_heads, conv_filters, dense_units, learning_rate):
    model = build_model(sequence_length, int(lstm_units), int(gru_units), int(transformer_heads), int(conv_filters), int(dense_units), learning_rate)
    
    history = model.fit(
        [train_sequences[:,:,0], train_sequences[:,:,1], train_sequences[:,:,2],
         train_sequences[:,:,0], train_sequences[:,:,1], train_sequences[:,:,2],
         train_sequences[:,:,0], train_sequences[:,:,1], train_sequences[:,:,2],
         train_sequences[:,:,0], train_sequences[:,:,1], train_sequences[:,:,2]],
        {'next_event': train_targets[0], 'agent': train_targets[1], 'context': train_targets[2], 'anomaly': train_targets[3]},
        validation_data=(
            [val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2],
             val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2],
             val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2],
             val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2]],
            {'next_event': val_targets[0], 'agent': val_targets[1], 'context': val_targets[2], 'anomaly': val_targets[3]}
        ),
        epochs=10, batch_size=64, verbose=0  # Set epochs to a lower value for faster tuning
    )

    # Evaluate the model
    eval_results = model.evaluate(
        [val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2],
         val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2],
         val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2],
         val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2]],
        {'next_event': val_targets[0], 'agent': val_targets[1], 'context': val_targets[2], 'anomaly': val_targets[3]}
    )
    return -eval_results[0]  # Return negative validation loss for maximization

# Define hyperparameter bounds
pbounds = {
    'lstm_units': (64, 256),
    'gru_units': (64, 256),
    'transformer_heads': (2, 8),
    'conv_filters': (64, 256),
    'dense_units': (64, 256),
    'learning_rate': (1e-5, 1e-2)
}

# Initialize Bayesian Optimization
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    verbose=2,
    random_state=1
)

# Perform Bayesian Optimization
optimizer.maximize(init_points=5, n_iter=10)

# Print the best hyperparameters
print("Best Hyperparameters:", optimizer.max)

# Train the final model with the best hyperparameters
best_params = optimizer.max['params']
model = build_model(
    sequence_length,
    int(best_params['lstm_units']),
    int(best_params['gru_units']),
    int(best_params['transformer_heads']),
    int(best_params['conv_filters']),
    int(best_params['dense_units']),
    best_params['learning_rate']
)

history = model.fit(
    [train_sequences[:,:,0], train_sequences[:,:,1], train_sequences[:,:,2],
     train_sequences[:,:,0], train_sequences[:,:,1], train_sequences[:,:,2],
     train_sequences[:,:,0], train_sequences[:,:,1], train_sequences[:,:,2],
     train_sequences[:,:,0], train_sequences[:,:,1], train_sequences[:,:,2]],
    {'next_event': train_targets[0], 'agent': train_targets[1], 'context': train_targets[2], 'anomaly': train_targets[3]},
    validation_data=(
        [val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2],
         val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2],
         val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2],
         val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2]],
        {'next_event': val_targets[0], 'agent': val_targets[1], 'context': val_targets[2], 'anomaly': val_targets[3]}
    ),
    epochs=50, batch_size=64
)

history_df = pd.DataFrame(history.history)
history_df.to_csv('training_results/RNN_Combine_model_GRU_Bayesian.csv', index=False)
# Evaluate the final model
eval_results = model.evaluate(
    [test_sequences[:,:,0], test_sequences[:,:,1], test_sequences[:,:,2],
     test_sequences[:,:,0], test_sequences[:,:,1], test_sequences[:,:,2],
     test_sequences[:,:,0], test_sequences[:,:,1], test_sequences[:,:,2],
     test_sequences[:,:,0], test_sequences[:,:,1], test_sequences[:,:,2]],
    {'next_event': test_targets[0], 'agent': test_targets[1], 'context': test_targets[2], 'anomaly': test_targets[3]}
)
print(f"Test Loss and Accuracy: {eval_results}")


|   iter    |  target   | conv_f... | dense_... | gru_units | learni... | lstm_u... | transf... |
-------------------------------------------------------------------------------------------------
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - agent_accuracy: 1.0000 - anomaly_accuracy: 0.5287 - context_accuracy: 0.4556 - loss: 0.7663 - next_event_accuracy: 1.0000
| 1         | -0.7571   | 144.1     | 202.3     | 64.02     | 0.00303   | 92.18     | 2.554     |
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - agent_accuracy: 1.0000 - anomaly_accuracy: 0.4630 - context_accuracy: 0.4732 - loss: 0.7582 - next_event_accuracy: 1.0000
| 2         | -0.759    | 99.76     | 130.3     | 140.2     | 0.005393  | 144.5     | 6.111     |
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - agent_accuracy: 1.0000 - anomaly_accuracy: 0.5744 - context_accuracy: 0.4450 - loss: 0.8025 - next_event_accuracy: 1.0000
| 3         | -0.8026   | 103.3     | 232.6     | 69.26     | 0.006708  | 144.1     | 5.352     |
5/5 ━━━━━━━━━━━━━━━━━━━━

In [2]:
num_samples = 10
sample_sequences = test_sequences[:num_samples]

# Predict the next values
predictions = model.predict(
    [sample_sequences[:,:,0], sample_sequences[:,:,1], sample_sequences[:,:,2],
     sample_sequences[:,:,0], sample_sequences[:,:,1], sample_sequences[:,:,2],
     sample_sequences[:,:,0], sample_sequences[:,:,1], sample_sequences[:,:,2]]
)

# Extract predictions
next_event_predictions = np.argmax(predictions[0], axis=-1)
agent_predictions = np.argmax(predictions[1], axis=-1)
context_predictions = np.argmax(predictions[2], axis=-1)
anomaly_predictions = (predictions[3] > 0.5).astype(int)

# Decode predictions
def decode_predictions(predictions, id_to_label):
    return [id_to_label[idx] for idx in predictions]

decoded_next_event_predictions = decode_predictions(next_event_predictions, event_type_mapping)
decoded_agent_predictions = decode_predictions(agent_predictions, agent_id_mapping)
decoded_context_predictions = decode_predictions(context_predictions, context_mapping)

# Decode input sequences
def decode_sequences(sequences, event_mapping, agent_mapping, context_mapping):
    decoded_sequences = []
    for seq in sequences:
        decoded_seq = []
        for step in seq:
            decoded_step = [
                event_mapping[step[0]],
                agent_mapping[step[1]],
                context_mapping[step[2]]
            ]
            decoded_seq.append(decoded_step)
        decoded_sequences.append(decoded_seq)
    return np.array(decoded_sequences)

decoded_sample_sequences = decode_sequences(sample_sequences, event_type_mapping, agent_id_mapping, context_mapping)

# Print decoded input sequences and predictions
for i in range(num_samples):
    print(f"Input sequence (event_id, agent_id, context):\n{decoded_sample_sequences[i]}")
    print(f"Predicted next event_id: {decoded_next_event_predictions[i]}")
    print(f"Predicted next agent_id: {decoded_agent_predictions[i]}")
    print(f"Predicted next context: {decoded_context_predictions[i]}")
    print(f"Predicted anomaly: {anomaly_predictions[i]}")
    print("-" * 30)

ValueError: Layer 'functional_31' expected 12 input(s). Received 9 instead.

In [ ]:
import numpy as np

def get_user_input(prompt, valid_options):
    while True:
        user_input = input(prompt).strip()
        if user_input in valid_options:
            return user_input
        print(f"Invalid input. Please choose from: {', '.join(valid_options)}")

def play_game(model, event_type_mapping, agent_id_mapping, context_mapping, sequence_length=10):
    sequence = []
    
    print("Let's start the game!")
    print(f"You'll provide the context for each step. The model will predict the event and agent.")
    print(f"We'll play for {sequence_length} steps.")
    
    for step in range(sequence_length):
        print(f"\nStep {step + 1}:")
        
        # Get context input from user
        context_options = list(context_mapping)
        context = get_user_input("Enter the context: ", context_options)
        context_id = np.where(context_mapping == context)[0][0]
        
        # If we don't have enough history, use placeholder values
        if len(sequence) < 3:
            event_id = 0
            agent_id = 0
        else:
            # Prepare input for model
            model_input = np.array(sequence[-3:])
            model_input = np.expand_dims(model_input, axis=0)
            
            # Make predictions
            predictions = model.predict([model_input[:,:,0], model_input[:,:,1], model_input[:,:,2],
                                         model_input[:,:,0], model_input[:,:,1], model_input[:,:,2],
                                         model_input[:,:,0], model_input[:,:,1], model_input[:,:,2]])
            
            # Extract predictions
            event_id = np.argmax(predictions[0][0])
            agent_id = np.argmax(predictions[1][0])
        
        # Decode predictions
        predicted_event = event_type_mapping[event_id]
        predicted_agent = agent_id_mapping[agent_id]
        
        print(f"Model predicts:")
        print(f"Event: {predicted_event}")
        print(f"Agent: {predicted_agent}")
        
        # Add to sequence
        sequence.append([event_id, agent_id, context_id])
    
    print("\nGame over! Thanks for playing.")

# Start the game
play_game(model, event_type_mapping, agent_id_mapping, context_mapping)

Let's start the game!
You'll provide the context for each step. The model will predict the event and agent.
We'll play for 5 steps.

Step 1:
